In [ ]:
# Import required libraries
!pip3 install dash
import pandas as pd
import dash
#import dash_html_components as html
#import dash_core_components as dcc
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px


# Read the spacex data into pandas dataframe
#spacex_df = pd.read_csv("spacex_launch_dash.csv")
spacex_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv', 
                            encoding = "ISO-8859-1")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

sitegrp_df = spacex_df.groupby('Launch Site').first()
sitegrp_df.reset_index(inplace=True)


# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                             options=[
                                                 {'label': 'All Sites', 'value': 'ALL'},
                                                 {'label': sitegrp_df.iloc[0,0], 'value': sitegrp_df.iloc[0,0]},
                                                 {'label': sitegrp_df.iloc[1,0], 'value': sitegrp_df.iloc[1,0]},
                                                 {'label': sitegrp_df.iloc[2,0], 'value': sitegrp_df.iloc[2,0]},
                                                 {'label': sitegrp_df.iloc[3,0], 'value': sitegrp_df.iloc[3,0]}
                                             ],
                                             value = 'ALL',
                                             placeholder = "Select a SpaceX Launch Site:",
                                             searchable = True
                                             ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',
                                                     1000: '1000',
                                                     5000: '5000',
                                                     7500: '7500'},
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values = 'class', names = 'Launch Site', title ='Success Rates for Launch Sites', hole = .3)
        fig.update_traces(textposition='inside', textinfo='percent+label')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df.loc[spacex_df['Launch Site'] == entered_site]
        fig = px.pie(filtered_df, names = 'class',  title = 'Success Rates for Launch Site ' + entered_site, hole = .3)
        fig.update_traces(textposition='inside', textinfo='percent+label')
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value'))
def get_scatter_chart(entered_site,entered_payload):
    low, high = entered_payload
    if entered_site == 'ALL':
        mask = (spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)
        fig = px.scatter(spacex_df[mask], x="Payload Mass (kg)", y="class", color="Booster Version Category", title='Correlation between Payload and Success Rate for Sites')
        return fig
    else:
        # return the outcomes scatter chart for a selected site
        filtered_df = spacex_df.loc[spacex_df['Launch Site'] == entered_site]
        mask = (filtered_df['Payload Mass (kg)'] > low) & (filtered_df['Payload Mass (kg)'] < high)
        fig = px.scatter(filtered_df[mask], x="Payload Mass (kg)", y="class", color="Booster Version Category", title='Correlation between Payload and Success Rate for Site ' + entered_site)
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()


     |████████████████████████████████| 9.6 MB 24.7 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 11.0 MB/s eta 0:00:01
     |████████████████████████████████| 357 kB 63.0 MB/s eta 0:00:01
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
